# 5: ALP processes

In [1]:
import alpaca
import numpy as np

mB = 5.279

We calculate the branching ratios for several processes: $a\to \eta\pi\pi$ (a hadronic ALP decay), $B^0 \to K^{*0} a$ (a mesonic decay into an ALP), and $B^+ \to K^+ a \to e^+ e^-$ (a decay mediated by an on-shell ALP):

In [10]:
transitions = [
    'a -> eta pi+ pi-',
    'B0 -> K*0 a',
    'B+ -> K+ e e',
]

couplings = alpaca.ALPcouplings(
{'cqL':1.0}, 
scale = 4*np.pi*1000, 
basis='derivative_above').match_run(
scale_out=mB,
basis='VA_below')

for tr in transitions:
    br = alpaca.branching_ratio(
        transition = tr,
        ma = 1.2,
        couplings = couplings,
        fa = 1000.0
    )
    print(f"BR({tr}) = {br:.5e}")

BR(a -> eta pi+ pi-) = 3.05992e-01
BR(B0 -> K*0 a) = 1.13747e-01
BR(B+ -> K+ e e) = 7.13931e-08


Next, we check how the branching ratio of $B^+\to K^+ a \to e^+ e^-$ if the ALP can decay into a dark sector with $\mathrm{BR}(a\to\mathrm{DS}) = 0.1$

In [15]:
br_ds = alpaca.branching_ratio(
        transition = transitions[2],
        ma = 1.2,
        couplings = couplings,
        fa = 1000.0,
        br_dark = 1e-1
    )
print(f"BR({transitions[2]}) with dark sector = {br_ds:.5e}")

BR(B+ -> K+ e e) with dark sector = 6.29901e-08


With a similar syntax, we can also calculate the decay width of the ALP and the modification of mesonic decay widths, and cross-sections:

In [3]:
dw_alp = alpaca.decay_width(
    'a',
    ma = 1.2,
    couplings = couplings,
    fa = 1000.0
)
print(f"Decay width of the ALP: {dw_alp:.5e} GeV")

dw_B = alpaca.decay_width(
    'B0',
    ma = 1.2,
    couplings = couplings,
    fa = 1000.0
)
print(f"Modification of the B0 decay width: {dw_B:.5e} GeV")

sigma_belleII = alpaca.cross_section(
    'e e -> gamma a',
    ma = 1.2,
    couplings = couplings,
    fa = 1000.0,
    s = 10.58**2
)
print(f"Cross-section at Belle II: {sigma_belleII:.5e} pb")

Decay width of the ALP: 2.18754e-10 GeV
Modification of the B0 decay width: 6.28133e-14 GeV
Cross-section at Belle II: 9.00771e-06 pb


These functions are vectorized, and accept grids of parameters as arguments:

In [4]:
# Create ranges of ma and fa, logarithmically spaced
ma = np.logspace(-1, 1, 30)
fa = np.logspace(3, 8, 30)
Lambda = 4*np.pi*fa
# Create a list of ALPcouplings
couplings = [
    alpaca.uvmodels.QED_DFSZ.get_couplings(
        {alpaca.uvmodels.beta: np.pi/4}, l
    ).match_run(10.0, 'VA_below') for l in Lambda
]
# Combine them into grids
x_ma, y_fa = np.meshgrid(ma, fa)
x_ma, y_couplings = np.meshgrid(ma, couplings)

dw_photons = alpaca.decay_width(
    'a -> gamma gamma',
    ma = x_ma,
    couplings = y_couplings,
    fa = y_fa,
    integrator = 'leadinglog'
)

In [5]:
dw_photons.shape

(30, 30)

In [6]:
max_index = np.unravel_index(np.argmax(dw_photons), dw_photons.shape)
print(f"Max decay width (a -> gamma gamma): {np.max(dw_photons):.5e} GeV, at ma = {x_ma[max_index]:.5} GeV, fa = {y_fa[max_index]:.5e} GeV")

Max decay width (a -> gamma gamma): 4.88730e-11 GeV, at ma = 7.279 GeV, fa = 1.00000e+03 GeV


In [7]:
dw_alp = alpaca.alp_channels_decay_widths(
    ma = x_ma[5,14],
    couplings = y_couplings[5,14],
    fa = y_fa[5,14]
)

br_alp = alpaca.alp_channels_branching_ratios(
    ma = x_ma[5,14],
    couplings = y_couplings[5,14],
    fa = y_fa[5,4]
)

In [8]:
dw_alp

{'a -> electron electron': array(2.36782499e-16),
 'a -> muon muon': array(9.86328411e-12),
 'a -> tau tau': array(0.),
 'a -> electron muon': array(0.),
 'a -> electron tau': array(0.),
 'a -> muon tau': array(0.),
 'a -> charm charm': array(0.),
 'a -> bottom bottom': array(0.),
 'a -> pion pion pion': array(4.34974225e-12),
 'a -> pion+ pion- pion0': array(4.23538608e-12),
 'a -> pion0 pion0 pion0': array(2.15177884e-13),
 'a -> eta pion pion': array(1.85198529e-14),
 'a -> eta pion0 pion0': array(7.11078076e-15),
 'a -> eta pion+ pion-': array(1.18182989e-14),
 'a -> eta_prime pion pion': array(0.),
 'a -> eta_prime pion0 pion0': array(0.),
 'a -> eta_prime pion+ pion-': array(0.),
 'a -> photon pion pion': array(3.80416846e-13),
 'a -> omega omega': array(0.),
 'a -> gluon gluon': array(0.),
 'a -> photon photon': array(2.22859409e-14),
 'a -> hadrons': array(4.74867895e-12),
 'a -> dark': array(0.)}

In [9]:
br_alp

{'a -> electron electron': array(1.61797621e-05),
 'a -> muon muon': array(0.67397545),
 'a -> tau tau': array(0.),
 'a -> electron muon': array(0.),
 'a -> electron tau': array(0.),
 'a -> muon tau': array(0.),
 'a -> charm charm': array(0.),
 'a -> bottom bottom': array(0.),
 'a -> pion pion pion': array(0.29722549),
 'a -> pion+ pion- pion0': array(0.28941134),
 'a -> pion0 pion0 pion0': array(0.01470348),
 'a -> eta pion pion': array(0.00126549),
 'a -> eta pion0 pion0': array(0.00048589),
 'a -> eta pion+ pion-': array(0.00080757),
 'a -> eta_prime pion pion': array(0.),
 'a -> eta_prime pion0 pion0': array(0.),
 'a -> eta_prime pion+ pion-': array(0.),
 'a -> photon pion pion': array(0.02599455),
 'a -> omega omega': array(0.),
 'a -> gluon gluon': array(0.),
 'a -> photon photon': array(0.00152284),
 'a -> hadrons': array(0.32448554),
 'a -> dark': array(0.)}